In [1]:
import numpy 
import pandas 
import tensorflow 
import nltk
import gensim

print(numpy.__version__)
print(pandas.__version__)
print(tensorflow.__version__)
print(nltk.__version__)
print(gensim.__version__)

1.21.4
1.3.3
2.6.0
3.6.5
4.1.2


# 데이터 전처리
## Preprocess Text
> Preprocess text
> - Convert English text (only) to lower
> - Remove the special characters

## Tokenization

In [2]:
import re

def preprocess_text(text: str) -> str:
    # Convert English text to lowercase
    if text.isascii():
        text = text.lower()

    # Remove all characters except Korean, English, numbers, and important special characters
    text = re.sub(r"[^ㄱ-ㅎㅏ-ㅣ가-힣a-zA-Z0-9!@#$%^&*()_+,\-.;: ]+", "", text)

    return text

In [3]:
from konlpy.tag import Mecab

In [4]:
def build_corpus(source, target, length = 20):
    mecab = Mecab()
    source = [preprocess_sentence(s) for s in source]
    target = [preprocess_sentence(t) for t in target]
    source_tokenized = [mecab.morphs(s) for s in source]
    target_tokenized = [mecab.morphs(t) for t in target]
    filtered_source = []
    filtered_target = []
    # what length? -> length
    for s, t in zip(source_tokenized, target_tokenized):
        if len(s) <= length and len(t) <= length:
            # remove duplicates
            if s not in filtered_source and t not in filtered_target:
                filtered_source.append(s)
                filtered_target.append(t)
    return filtered_source, filtered_target

# Step 4. Augmentation
다운로드한 모델을 활용해 데이터를 Augmentation 하세요! 앞서 정의한 lexical_sub() 함수를 참고하면 도움이 많이 될 겁니다.

Augmentation된 que_corpus 와 원본 ans_corpus 가 병렬을 이루도록, 이후엔 반대로 원본 que_corpus 와 Augmentation된 ans_corpus 가 병렬을 이루도록 하여 전체 데이터가 원래의 3배가량으로 늘어나도록 합니다.

- Lexical substitution 
    - randomness

In [1]:
from gensim.models import Word2Vec

def lexical_sub(sentence, word_vec):
    # 임베딩 모델 로드
    model = Word2Vec.load('path/to/word2vec/model')

    # 입력 문장을 임베딩으로 확장
    embedded_sentence = [model.wv[word] for word in sentence.split()]
    
    # 대체할 단어 선택
    selected_w = random.choice(sentence)

    # 대체할 단어와 가장 유사한 단어 찾기
    similar_word = model.wv.most_similar(target_word, topn=1)[0][0]

    # 선택한 단어로 대체
    substituted_sentence = sentence.replace(target_word, similar_word)

    return substituted_sentence

# Step 6. 훈련하기
앞서 번역 모델을 훈련하며 정의한 Transformer 를 그대로 사용하시면 됩니다! 대신 데이터의 크기가 작으니 하이퍼파라미터를 튜닝해야 과적합을 피할 수 있습니다. 모델을 훈련하고 아래 예문에 대한 답변을 생성하세요! 가장 멋진 답변과 모델의 하이퍼파라미터를 제출하시면 됩니다.

# Step 7. 성능 측정하기
- 챗봇의 경우, 올바른 대답을 하는지가 중요한 평가 지표입니다. 올바른 답변을 하는지 눈으로 확인할 수 있겠지만, 많은 데이터의 경우는 모든 결과를 확인할 수 없을 것입니다. 주어진 질문에 적절한 답변을 하는지 확인하고, BLEU Score를 계산하는 calculate_bleu() 함수도 적용해 보세요.